In [ ]:
# Testing Cell
from aviary.interface.cmd_entry_points import _command_map
from aviary.utils.doctape import glue_keys, glue_variable

current_glued_vars = []

# glue all the options of 'aviary'
glue_keys(_command_map)
for key in _command_map:
    glue_variable('aviary ' + key, md_code=True)

# Command Line Tools

Aviary has a number of command line tools that are available via the `aviary`
command.

```{note}
When using a command line tool on a script that takes its own command line arguments, those
arguments must be placed after a `--` on the command line.  Anything to the right of the
`--` will be ignored by the Aviary command line parser and passed on to the user script.
For example: `Aviary sub_command -o foo.html myscript.py -- -x --myarg=bar` would pass
`-x` and `--myarg=bar` as args to `myscript.py`.
```

All available `aviary` sub-commands can be shown using the following command:

```
aviary -h
```

In [ ]:
!aviary -h 2>/dev/null

To get further info on any sub-command, follow the command with a *-h*.  For example:

```
aviary run_mission -h
```

In [ ]:
!aviary run_mission -h 2>/dev/null

## Available Commands and Their Usage


(aviary-run_mission-command)=
### aviary run_mission

{glue:md}`run_mission` is a command line interface that will run an analysis on a given csv input file.

```
aviary run_mission -h
```

In [ ]:
!aviary run_mission -h 2>/dev/null

To use {glue:md}`small_single_aisle_GASP.csv`, run the command
{glue:md}`aviary run_mission models/aircraft/small_single_aisle/small_single_aisle_GASP.csv` in
{glue:md}`aviary/models/aircraft/small_single_aisle`

SNOPT is the default optimizer, but IPOPT is available as well.

In [ ]:
# Testing Cell
import os
import subprocess

import aviary.api as av
from aviary.utils.doctape import glue_variable

str_folder = 'models/aircraft/small_single_aisle'
folder = av.get_path(str_folder)
model_dir = folder.relative_to(av.top_dir.parent)
glue_variable(model_dir, md_code=True)

str_model = 'small_single_aisle_GASP.csv'
folder = av.get_path(str_folder) / (str_model)
file_name = os.path.basename(folder)
glue_variable(file_name, md_code=False)

command = 'aviary run_mission ' + str_folder + '/' + str_model
glue_variable(command, md_code=True)
command += ' --max_iter 0 --optimizer IPOPT'  # max_iter to limit build time, IPOPT to run on CI
subprocess.run(command.split()).check_returncode()

In [ ]:
# Testing Cell
import inspect

import aviary.api as av
from aviary.interface.methods_for_level1 import run_level_1
from aviary.utils.doctape import glue_actions, glue_variable
from aviary.variable_info.variables import Settings

# glue all the options of 'aviary run_mission'
glue_actions('run_mission', current_glued_vars, md_code=True)

glue_variable(av.EquationsOfMotion.HEIGHT_ENERGY.value, md_code=False, display=True)
glue_variable(av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.value, md_code=False, display=True)
glue_variable(av.EquationsOfMotion.SOLVED_2DOF.value, md_code=False, display=True)
glue_variable('equations_of_motion', Settings.EQUATIONS_OF_MOTION, display=True)

# obtain the default value of maximum number of iterations from function run_level_1().
max_iter = inspect.signature(run_level_1).parameters['max_iter'].default
if 'max_iter' not in current_glued_vars:
    glue_variable('max_iter', str(max_iter), md_code=False)
    current_glued_vars.append('max_iter')

{glue:md}`input_deck` is the path to vehicle input deck .csv file.
{glue:md}`--optimizer` is the name of the optimizer. The default is `IPOPT`.
{glue:md}`--phase_info` is the path to phase info file. If it is missing, it depends on the mission method (`equations_of_motion` with value of {glue:md}`2DOF` or {glue:md}`height_energy`) which is defined in the .csv input file.
{glue:md}`--max_iter` is the maximum number of iterations. The default is {glue:md}`max_iter`.

More detailed information and examples can be found in the [Level 1 interface](../getting_started/onboarding_level1.ipynb).

In [ ]:
# Testing Cell
from aviary.utils.doctape import glue_actions

# glue all the options of 'aviary fortran_to_aviary'
glue_actions('fortran_to_aviary', current_glued_vars, md_code=True)

(aviary-fortran_to_aviary-command)=
### aviary fortran_to_aviary

The {glue:md}`aviary fortran_to_aviary` command will convert a Fortran input deck to an Aviary csv.



```
aviary fortran_to_aviary -h
```

In [ ]:
!aviary fortran_to_aviary -h 2>/dev/null

The only two required inputs are the filepath to the input deck and {glue:md}`-l` (for {glue:md}`--legacy_code` with options `FLOPS` and `GASP`).
When this command is run, a brief message is printed. To print more messages, one can set verbosity level higher. For example, `-v 3` will result in debug messages being printed. See [Controlling Display Levels](../developer_guide/coding_standards.ipynb) for more details.
If an invalid filepath is given, pre-packaged resources will be checked for input decks with a matching name.
If the output file name is not specified, a detault name is assumed to be the trunk of the input file name with `csv` as file extension. For example, an input file `sample.dat` will result in `sample_converted.csv`.
If the output file exists, the command will not run unless the user specifies {glue:md}`--force` to force the overwritten action.

Here, pre-packaged resources are absolute path, relative path, Aviary based path, and relative path relative to the Aviary models folder.

Notes for input decks:
- When specifying variables from FORTRAN, they should be in the appropriate NAMELIST
- Names are not case-sensitive
- Comments can be added using "!"
- Lists can be entered by separating values with commas on the same line
- For GASP variables where required, individual list elements can be specified by adding an index after the variable name
    - (NOTE: 1 indexing is used inside NAMELISTS, while 0 indexing is used outside NAMELISTS)

Example inputs (GASP-style file):
```
$INGASP
DELP = 7.34797 ! typical GASP input
ARNGE(1) = 3600 ! variable with specified index
ACLS = 0.0,0.40,0.60,0.80,1.00,1.20, 1.40,1.60,1.80, ! values given as array
```

Example usage:
`aviary fortran_to_aviary --legacy_code GASP --force GASP_test.dat` Converts the GASP input deck to Aviary (even if a file with the name GASP_test.dat already exists).

In [ ]:
# Testing Cell

# glue all the options of 'aviary convert_engine'
glue_actions('convert_engine', current_glued_vars, md_code=True)

(aviary-hangar-command)=
### aviary hangar

The {glue:md}`aviary hangar` command will copy files and folders from the Aviary hangar to an accessible directory.
This is useful for users that have pip installed Aviary, but want to experiment with the included examples.




```
aviary hangar -h
```

In [ ]:
!aviary hangar -h 2>/dev/null

The only required input is the name of an input deck.
This can be specified as the name of the file, the path from [aviary/models](https://github.com/OpenMDAO/Aviary/tree/main/aviary/models), the name of a folder in aviary/models. Multiple files and folders can be copied at once.
Optionally, the output directory can be specified; if it is not, the files will be copied into the current working directory in a folder called `aviary_models`.
If specified, the output directory will be created as needed.

Example usage:
```
# Copy all files in the engines folder
aviary hangar engines
# Copy the 22k turbofan deck
aviary hangar turbofan_22k.txt
# Copy the N3CC data
aviary hangar advanced_single_aisle/advanced_single_aisle_data.py
# Copy the Fortran and Aviary input decks for the small single aisle
aviary hangar small_single_aisle_GwGm.dat small_single_aisle_GwGm.csv
# Copy the engine model into ~/example_files
aviary hangar turbofan_22k.txt -o ~/example_files
```

In [ ]:
# Testing Cell
import os
import subprocess
import tempfile

commands = [
    'engines',
    'turbofan_22k.csv',
    'advanced_single_aisle/advanced_single_aisle_data.py',
    'small_single_aisle_GASP.dat small_single_aisle_GASP.csv',
    'turbofan_22k.csv -o ~/example_files',
]
# Save the current directory
original_cwd = os.getcwd()

try:
    with tempfile.TemporaryDirectory() as tempdir:
        os.chdir(tempdir)
        for command in commands:
            command = 'aviary hangar ' + command
            subprocess.run(command.split()).check_returncode()
finally:
    # Always restore the original directory
    os.chdir(original_cwd)

(aviary-EDC-command)=
### aviary convert_engine

The {glue:md}`aviary convert_engine` command will convert a legacy formatted (FLOPS or GASP) engine deck into an Aviary formatted engine deck.

```
aviary convert_engine -h
```

In [ ]:
!aviary convert_engine -h 2>/dev/null

Users must provide the path or name of an input deck, and the engine format being converted.
If an invalid filepath is given for the input file, pre-packaged resources will be checked for input decks with a matching name.

The path to the output file name is optional. If it is missing, the output file name takes the 
trunk of the input file name with `deck` as default file extension. For example, an input file `sample.eng` will result in `sample.csv` unless the user specifies the output file name.

If the output file exists, it will be overwritten.

The engine format is specified by {glue:md}`-f` or {glue:md}`--data_format` with one of `FLOPS`, `GASP`, and `GASP_TS` string (`TS` stands for turboshaft). If multiple are specified, the last one will be used.

Notes for input decks:
- Turbofan decks for both FLOPS and GASP can be converted
- Turboshaft decks for GASP can also be converted
- Comments can be added using #


Example usage:

```
# Convert a GASP based turbofan
aviary convert_engine turbofan_23k_1.eng turbofan_23k_1.csv -f GASP 
# Convert a FLOPS based turbofan
aviary convert_engine turbofan_22k.eng turbofan_22k.csv -f FLOPS 
# Convert a GASP based turboshaft
aviary convert_engine turboshaft_4465hp.eng turboshaft_4465hp.csv --data_format GASP_TS
```

In [ ]:
# Testing Cell
from aviary.utils.doctape import run_command_no_file_error

commands = [
    'utils/test/data/turbofan_23k_1.eng turbofan_23k_1.csv -f GASP',
    'utils/test/data/turbofan_22k.txt turbofan_22k.csv -f FLOPS',
    'utils/test/data/turboshaft_4465hp.eng turboshaft_4465hp.csv -f GASP_TS',
]
for command in commands:
    run_command_no_file_error('aviary convert_engine ' + command)

In [ ]:
# Testing Cell

# glue all the options of 'aviary convert_aero_table'
glue_actions('convert_aero_table', current_glued_vars, md_code=True)

(aviary-ATC-command)=
### aviary convert_aero_table

The {glue:md}`aviary convert_aero_table` command will convert a legacy formatted (FLOPS or GASP) aero table into an Aviary formatted aero table.


```
aviary convert_aero_table -h
```

In [ ]:
!aviary convert_aero_table -h 2>/dev/null

Users must provide the path or name of an input deck and the data format being converted.
Optionally, the path to the output file can also be specified, otherwise the default output file will be in the same location and have the same name as input file, but with '_aviary' appended to the end of filename trunk while the file extension is preserved. For example, an input file `sample.txt` will result in `sample_aviary.txt` unless the user specifies the output file name.
If both {glue:md}`-f` and {glue:md}`--data_format` are specified, the later one is taken.
If an existing file has the same name as output file name, the existing file will be overwritten.
If an invalid filepath is given for the input file, pre-packaged resources will be checked for input decks with a matching name.

Notes for input decks:
- Aero tables for both FLOPS and GASP can be converted
- GASP tables will create a single file containing all the lift and drag information
- FLOPS tables will create two files, one containing the lift-dependent drag and the other containing the lift-independent drag.
- Comments can be added using #


Example usage:
```
# Converts a GASP based aero table
aviary convert_aero_table -f GASP subsystems/aerodynamics/gasp_based/data/GASP_aero_free.txt large_single_aisle_1_aero_flaps.txt
# Converts a FLOPS based aero table
aviary convert_aero_table -f FLOPS utils/test/flops_test_polar.txt aviary_flops_polar.txt
```

In [ ]:
# Testing Cell
from aviary.utils.doctape import run_command_no_file_error

commands = [
    '-f GASP subsystems/aerodynamics/gasp_based/data/GASP_aero_free.txt large_single_aisle_1_aero_flaps.txt',
    '-f FLOPS utils/test/flops_test_polar.txt aviary_flops_polar.txt',
]
for command in commands:
    run_command_no_file_error('aviary convert_aero_table ' + command)

(aviary-PMC-command)=
### aviary convert_prop_table

The {glue:md}`aviary convert_prop_table` command converts a legacy formatted (GASP) propeller map into an Aviary formatted propeller map. Currently, GASP is the only format supported.

```
aviary convert_prop_table -h
```

In [ ]:
!aviary convert_prop_table -h 2>/dev/null

Users must provide the path or name of an input deck, and optionally the path to the desired output file. If not provided, the output file will have the same filename as the input file but with the exension `.prop`. This is arbitrary, and `.prop` is not a required file extension for propeller data files, but it is designed to help users recognize the purpose of the data table without needing to open it.
If an invalid filepath is given for the input file, Aviary's files will be checked for input decks with a matching name.

Notes for input decks:
- There are two options for Mach number: Mach number and helical Mach number at 75% radius. We provide one example for each. This is the first integer on the first line of the input file. (1 = Mach, 2 = helical Mach)
- Comments can be added using #


Example usage:

```
# Convert GASP based propeller map PropFan.map to Aviary data table PropFan.csv
aviary convert_prop_table PropFan.map PropFan.csv
# Convert GASP based propeller map general_aviation.map to Aviary data table general_aviation.csv
aviary convert_prop_table general_aviation.map general_aviation.csv
# Convert GASP based propeller map general_aviation.map to Aviary data table general_aviation.csv
aviary convert_prop_table general_aviation.map
```
The first example uses Mach number and the second example uses helical Mach number. 
Note that the output file name was not specified in the third example.

In [ ]:
# Testing Cell
from aviary.utils.doctape import run_command_no_file_error

commands = [
    'models/engines/propellers/PropFan.map PropFan.csv',
    'models/engines/propellers/general_aviation.map general_aviation.csv',
    'models/engines/propellers/general_aviation.map',
]
for command in commands:
    run_command_no_file_error('aviary convert_prop_table ' + command)

In [ ]:
# glue all the options of 'aviary convert_prop_table'
glue_actions('convert_prop_table', current_glued_vars, md_code=True)

(aviary-draw-command)=
### aviary draw_mission

The {glue:md}`aviary draw_mission` command will bring up a new graphical interface for users to create a  mission. This command does not have an input. More details can be found at [Drawing and running simple missions](drawing_and_running_simple_missions.ipynb).

(aviary-plot-command)=
### aviary plot_drag_polar

The {glue:md}`aviary plot_drag_polar` command will bring up a new graphical interface for users to draw drag polar. No options are needed on the command line but a file explorer window will pop-up to allow the user to select a drag polar file (a `.csv` file). It is not working at this time.

```
aviary plot_drag_polar -h
```

In [ ]:
!aviary plot_drag_polar -h 2>/dev/null

In [ ]:
# Testing Cell

# glue all the options of 'aviary dashboard'
glue_actions('dashboard', current_glued_vars, glue_default=True, md_code=True)

(aviary-dashboard-command)=
### aviary dashboard

The {glue:md}`aviary dashboard` command will bring up a dashboard that lets the user easily browse between the reports and files that are generated during an Aviary run.

```
aviary dashboard -h
```

In [ ]:
!aviary dashboard -h 2>/dev/null

To use this utility, either a problem has been run or a run script is provided.

{glue:md}`--problem_recorder` is an input. Default is {glue:md}`problem_recorder_default`.
{glue:md}`--driver_recorder` is an optional input.
{glue:md}`--port` is the dashboard server port ID. The default is {glue:md}`port_default` meaning any free port.
{glue:md}`-b` or {glue:md}`--background` indicates to run in background. Default is `False`.
{glue:md}`-d` or {glue:md}`--debug` indicates to show debugging output. Default is `False`.
{glue:md}`--save` is the name of zip file in which dashboard files are saved. If no argument given, use the script name to name the zip file.
{glue:md}`--force` indicates to overwrite the saved zip file. The default is `False`.
{glue:md}`script_name` is the name of aviary script that was run (not including .py), or the csv input filename if the user runs a Level 1 script.


More discussion on {glue:md}`aviary dashboard` command can be found in [Postprocessing and Visualizing Results from Aviary](postprocessing_and_visualizing_results.ipynb).